# Basic functions

In [34]:
def to_mont(a):
    return a * (2^storage_size)

def from_mont(a):
    return a / (2^storage_size)

def to_words(a):
    word = rz(2^64)
    aI = rz(a)
    res = []
    
    for i in range(len_in_words):
        lsw = aI % word
        res.append(lsw)
        aI -= lsw
        aI /= word
    
    if aI == 0:
        return res
    return []

def from_words(a):
    word = rz(2^64)
    res = k(0)
    pos = k(1)
    
    for aI in a:
        res += aI * pos
        pos *= word
    
    return res

def to_mont_words(a):
    aMont = to_mont(a)
    return to_words(aMont)


def from_mont_words(a):
    aMont = from_words(a)
    return from_mont(aMont)

#from termcolor import cprint

def print_values(line, names):
    #cprint("After line " + str(line), "red")
    print("After line " + str(line))
    for name in names.split():
        print(name, to_mont_words(globals()[name]), '\n')

def CMOV(a, b, c):
    if c:
        return b
    return a

def sqrt_ratio(u,v):
    r = u/v
    rt = r ^ ((p+1)/4)
    return (rt^2 == r, rt)  

def sgn0(a):
    return (rz(a) % 2) == 1

## Constants for bls12-381

In [ ]:
rz = IntegerRing()
size = 381
p = rz(0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab)
m = 2
k = GF(0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab)
Z = k(11)
A = k(0x144698a3b8e9433d693a02c96d4982b0ea985383ee66a8d8e8981aefd881ac98936f8da0e0f97f5cf428082d584c1d)
B = k(0x12e2908d11688030018b12e8753eee3b2016c1f0f24f4070a0b9c14fcef35ef55a23215a316ceaa5d1cc48e98e172be0)
storage_size = ( int(size/64) + 1) * 64

### sqrtRatio for 3 mod 4

In [9]:
u = k(3297453399026529404493391225421873154938040315357981303415954598599539802924806733815287302637231826557354531118851)

tv1 = u^2
tv1 = Z * tv1
tv2 = tv1^2
tv2 = tv2 + tv1
tv3 = tv2 + 1

print_values(5, "u tv1 tv2 tv3")

tv3 = B * tv3
tv4 = CMOV(Z, -tv2, tv2 != 0)
tv4 = A * tv4
tv2 = tv3^2
tv6 = tv4^2

print_values(10, "tv2 tv3 tv4 tv6")

tv5 = A * tv6
tv2 = tv2 + tv5
#print_values(12, "tv2 tv3")
tv2 = tv2 * tv3
#print_values(13, "tv2")
tv6 = tv6 * tv4
tv5 = B * tv6

print_values(15, "tv2 tv5 tv6")
    
tv2 = tv2 + tv5
x = tv1 * tv3
(is_gx1_square, y1) = sqrt_ratio(tv2, tv6)
y = tv1 * u
y = y * y1

print_values(20, "x y y1 tv2")
print("is_gx1_square", is_gx1_square)

x = CMOV(x, tv3, is_gx1_square)
y = CMOV(y, y1, is_gx1_square)
e1 = sgn0(u) == sgn0(y)
y = CMOV(-y, y, e1)
x = x / tv4

print_values(25, "x y")
print("e1", e1)

After line 5
u [941031641141724048, 7593419090796165139, 13447299832369701844, 7664570780628181207, 16833839340160123079, 332469494419187881] 

tv1 [3516142432136896772, 989984030340953070, 8323637589625304737, 14762897644741152998, 13972423264883779004, 700411725697946347] 

tv2 [18260259361604356633, 17011240093717171190, 12062012208352901412, 14141187874478462529, 17384135312051499430, 1259585366981427348] 

tv3 [13363157716792581995, 13356569052254637049, 11492242767550290875, 15534784565169543879, 156585674763579708, 968343264215580302] 

After line 10
tv2 [17384830729097079884, 14045349173857942013, 18295484079495378141, 14864088641379034595, 916614676115323181, 1827000547951513052] 

tv3 [3084661094584357932, 15065437943030972296, 8101785476161326048, 10309100984372636156, 8122376533328709437, 491500006511882507] 

tv4 [18267095155667468194, 9051927314548265906, 1345600663401514376, 6597902347854348425, 6642601737794002942, 1109251153611440862] 

tv6 [1245113284231550709, 176626

In [21]:
def sqrt_ratio_opt(u,v):
    tv1 = v^2
    tv2 = u * v
    tv1 = tv1 * tv2
    y1 = tv1^c1
    y1 = y1 * tv2
    y2 = y1 * c2
    tv3 = y1^2
    tv3 = tv3 * v
    isQR = tv3 == u
    y = CMOV(y2, y1, isQR)
    return (isQR, y)

In [24]:
print_values(-1, "c2")
print("c1 =", c1)
print("c1 =", to_words(c1),'\n')
qr, y = sqrt_ratio_opt(k(1),k(1))
print_values(0, "y")
print("qr =", qr)
print("y = 1?", y == k(1))

After line -1
c2 [17544630987809824292, 17306709551153317753, 8299808889594647786, 5930295261504720397, 675038575008112577, 167386374569371918] 

c1 = 1000602388805416848354447456433976039139220704984751971333014534031007912622709466110671907282253916009473568139946
c1 = [17185665809301629610, 552535377879302143, 15693976698673184137, 15644892545385841839, 10576397981472451381, 468449654411884966] 

After line 0
y [8505329371266088957, 17002214543764226050, 6865905132761471162, 8632934651105793861, 6631298214892334189, 1582556514881692819] 

qr = True
y = 1? True


In [ ]:
def testSqrtRatio()

In [33]:
u = from_mont_words([3752852834233450803, 10015304229637369378, 6482406239105581310, 1802624635905610022, 11716583840524549243, 1670704604553607051])
v = from_mont_words([16538149341274582162, 2654217574689430748, 4191868356445146499, 16611300210497698397, 10619697645702806389, 130786230622822284])
print("non mont u =", u, "v =", v)
print_values(-1, "u v")
print("c2^2 + Z =", c2^2 + Z)
negC2 = -c2
print("nonmont c2 =", c2, "negC2 =", negC2)
print_values(0, "negC2")

non mont u = 64698747985976059993142982927630677764380142939166895442303395594115446112008607578013245761275626776303342465549 v = 2231083048247663781863405666754311921963135649812079269756991928681364133099053904228640779353655192779108829213788
After line -1
u [3752852834233450803, 10015304229637369378, 6482406239105581310, 1802624635905610022, 11716583840524549243, 1670704604553607051] 

v [16538149341274582162, 2654217574689430748, 4191868356445146499, 16611300210497698397, 10619697645702806389, 130786230622822284] 

c2^2 + Z = 0
nonmont c2 = 674008237974212438248723729577393239183484355108025529917084410198223800758598092373770506194640977045288353413059 negC2 = 3328401317247454955169066096158510917373398464830982355414973725925807849732239772068917122934374686992605919146728
After line 0
negC2 [14304544101977590919, 3350176034073442437, 17582609757678985529, 1309042698909992113, 4737065203462589718, 1706412243078167948] 



### Constants for bls12-377

In [11]:
rz = IntegerRing()
size = 377
p = rz(0x01ae3a4617c510eac63b05c06ca1493b1a22d9f300f5138f1ef3622fba094800170b5d44300000008508c00000000001)
k = GF(p)
storage_size = ( int(size/64) + 1) * 64
Z = 5

qDiv = p - 1
c1 = 0
while qDiv % 2 == 0:
    qDiv /= 2
    c1+=1

c2 = (p - 1) / (2^c1)
c3 = (c2 - 1) / 2            # Integer arithmetic
c4 = 2^c1 - 1                # Integer arithmetic
c5 = 2^(c1 - 1)              # Integer arithmetic
c6 = k(Z)^c2
c7 = k(Z)^((c2 + 1) / 2)

In [14]:
for i in range(1,8):
    name = 'c' + str(i)
    print(name, '=', str(to_words(globals()[name])) + '\n')

c1 = [46, 0, 0, 0, 0, 0]

c2 = [8435453208297608227, 9853568280881552429, 7479357291536088013, 1657802422768920715, 16796279350917535980, 1720]

c3 = [13441098641003579921, 14150156177295552022, 12963050682622819814, 828901211384460357, 8398139675458767990, 860]

c4 = [70368744177663, 0, 0, 0, 0, 0]

c5 = [35184372088832, 0, 0, 0, 0, 0]

c6 = [13432934708832943080, 2206831062850032006, 14333447693842680516, 15082496647900565774, 10370064699282770698, 15812341292448861]

c7 = [1805497067151293688, 9839655818449531011, 9967078960026416217, 9646276585588320217, 9606226077609452738, 99149128060966092]



### sqrtRatio Test General

In [29]:
u = from_mont_words([15277767486155326275, 5422982723023692250, 3700681976081333390, 16318513938059080991, 14435898238372592577, 67639875291413723])
v = from_mont_words([4607951573955742566, 7838878090098306660, 17460363218501908074, 534496730432711842, 197058217151002387, 30296605951459846])

tv1 = c6
#del tv1
tv2 = v^c4
tv3 = tv2^2
tv3 = tv3 * v
tv5 = u * tv3
tv5 = tv5^c3
tv5 = tv5 * tv2
tv2 = tv5 * v
tv3 = tv5 * u
tv4 = tv3 * tv2

print_values(10, "tv1 tv2 tv3 tv4 tv5")

tv5 = tv4^c5
isQR = tv5 == 1
tv2 = tv3 * c7
tv5 = tv4 * tv1
tv3 = CMOV(tv2, tv3, isQR)
tv4 = CMOV(tv5, tv4, isQR)

for i in range(c1, 1, -1):
    tv5 = i - 2
    tv5 = 2^tv5
    tv5 = tv4^tv5
    e1 = tv5 == 1
    tv2 = tv3 * tv1
    tv1 = tv1 * tv1
    tv5 = tv4 * tv1
    tv3 = CMOV(tv2, tv3, e1)
    tv4 = CMOV(tv5, tv4, e1)

print(isQR, tv3)

if isQR and tv3^2 != u/v or not isQR and tv3^2 != u/v * Z:
    print("incorrect")

After line 10
tv1 [7563926049028936178, 2688164645460651601, 12112688591437172399, 3177973240564633687, 14764383749841851163, 52487407124055189] 

tv2 [48228029360928929, 15092953863106743314, 6983844333915777016, 10752422538179116303, 12103750235620913289, 1255219401708384] 

tv3 [4326175517770519301, 13331042191855875349, 506856078558015485, 3532786325944316907, 365834073121535883, 57630719645460388] 

tv4 [5214023629322226004, 13766855470125513985, 14146988209837136763, 1772468608803668238, 7218613592651140014, 27296696334549815] 

tv5 [350366515122944989, 4457338894683472227, 4259245614901703092, 12473093046791768647, 7898249447376005343, 74009396817483689] 

False 114988722959113475945166330707672081794762494895493993806724881275749988406473121447312556715993746771838909072442


### bw6-633 Constants

In [39]:
Z = 2
rz = IntegerRing()
size = 633
p = rz("20494478644167774678813387386538961497669590920908778075528754551012016751717791778743535050360001387419576570244406805463255765034468441182772056330021723098661967429339971741066259394985997")
q = p
k = GF(p)
len_in_words = int(size/64) + 1
storage_size = len_in_words * 64
c1 = (q - 5) / 8
c2 = -sqrt(k(-1))#to match golang
c3 = sqrt(k(Z) / c2)

c2Sq = c2^2
c3Sq = c3^2

if c2Sq != -1:
    print('Error: c2^2 =', c2Sq)

if c3Sq != Z / c2:
    print('Error: c3^2')

print('c1 =', to_words(c1))
print('c2 =', to_words(c2))
print('-c2 =', to_words(-c2))
print('c3 =', to_words(c3))

c1 = [11162491485216964609, 12080575573054258919, 18083845653922265156, 3441261431666226304, 1665016759523913609, 17305737805340333202, 16846610316412812101, 13051320776478636632, 9188261237022713174, 10357844467411948]
c2 = [12175174062465388638, 13382317189617221432, 153632921634065963, 16023698145664183155, 5360228551239893799, 8444433052832970306, 9682196376973769096, 13367273034960920244, 9082754551548314365, 61601480273317840]
-c2 = [3337781524432121775, 9475311099978643460, 15389923793777193977, 11506393307665627284, 7959905524951415073, 874260873922833998, 14410221712071418023, 17256316882029966354, 9083103123504736183, 21261275465977747]
c3 = [3337781524432121776, 9475311099978643460, 15389923793777193977, 11506393307665627284, 7959905524951415073, 874260873922833998, 14410221712071418023, 17256316882029966354, 9083103123504736183, 21261275465977747]
